In [1]:
from os import listdir
from os.path import isfile, join
import js2py
import urllib2
import json
from bs4 import BeautifulSoup
import csv
import pandas as pd
from pandas.io.json import json_normalize
import re
from  config import DATA_PATH,RAW_PATH

In [2]:
DATA_PATH = "comments/"
FT_ARTICLES_CSV = "ft_articles_link/"
URL_SEARCH_KEYWORDS = "https://www.ft.com/search?q="
KEY_WORDS = ['Syria','Gazprom','Brexit','MH17','NATO','Trump','Assad','Georgia','IOC','Doping','Putin','Merkel','Refugees','Afd','Le+Pen']
# KEY_WORDS = ['Obama','Russia',]

cleanr =re.compile('<.*?>')

NameError: name 're' is not defined

In [7]:
def cleanhtml(raw_html):
    if isinstance(raw_html, float):
        raw_html = str(raw_html)
    cleantext = re.sub(cleanr,'', raw_html)
    return cleantext

In [8]:
def save_to_csv(url_list,name):
    filename = name.split('/')[-1]
    filename = filename.split("=")[-1]
    with open(FT_ARTICLES_CSV+filename+'.csv','wb') as out:
        wr = csv.writer(out)
        for url in url_list:
            wr.writerow([url])

In [9]:
def save_livefyre_links(url,i):
    url +="&page="+str(i)
    urls_articles = []
    req = urllib2.Request(url)
    req.add_header('Referer', 'https://www.google.com/')
    req.add_header('User-Agent', 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)')
    resp = urllib2.urlopen(req)
    page = resp.read()
    if page is None:
        print "Page not found"+ url
        return
    soup = BeautifulSoup(page)
    error = soup.find_all('h1', {"class":"error-page__title"})
    if error is  None:
        print "Page not found :"+url
        return
    
    links = soup.find_all('a', {"class":"card__title"},href=True)
    if links is not None:
        urls_articles+=[x['href'].strip("/content/") for x in links]
    return urls_articles

In [10]:
def parse_ft_search_pages():
    keywords = [URL_SEARCH_KEYWORDS+x for x in KEY_WORDS]
    urls = keywords #+ URL_SPECIAL
    for url in urls:
        urls_articles = []
        i=1
        while True:
            try:
                print url+str(i)
                urls_articles += save_livefyre_links(url,i)
                i +=1
            except Exception as e:
                print e
                save_to_csv(urls_articles,url)
                break    

In [11]:
def create_single_url_list():
    df = pd.DataFrame()
    articles_csv_list = [f for f in listdir(FT_ARTICLES_CSV) if isfile(join(FT_ARTICLES_CSV, f))]
    for article in articles_csv_list:
        df1 = pd.read_csv(FT_ARTICLES_CSV+article,header=None)
        df1.columns =['ft_hash']
        df1['topic']= article.split(".")[0]
        df = pd.concat([df,df1],axis=0)
    df = df.drop_duplicates(['ft_hash'])
    df = df[df.ft_hash.str.contains("http") == False]
    df['livefyre_id']=df.ft_hash.apply(url_encoder.vb)
    df.to_csv("ft_id_with_livefyre_id.csv")

In [12]:
df = pd.read_csv("ft_id_with_livefyre_id.csv",index_col=0)
df.head()

,ft_hash,topic,livefyre_id
0,046c77e-8615-11e6-8897-2359a58ac7a5,Russia,MDQ2Yzc3ZS04NjE1LTExZTYtODg5Ny0yMzU5YTU4YWM3YTU.
1,df978a48-85f2-11e6-a29c-6e7d9515ad15,Russia,ZGY5NzhhNDgtODVmMi0xMWU2LWEyOWMtNmU3ZDk1MTVhZDE1
2,afb4972-8650-11e6-8897-2359a58ac7a5,Russia,YWZiNDk3Mi04NjUwLTExZTYtODg5Ny0yMzU5YTU4YWM3YTU.
3,59429afa-8647-11e6-8897-2359a58ac7a5,Russia,NTk0MjlhZmEtODY0Ny0xMWU2LTg4OTctMjM1OWE1OGFjN2E1
4,fabe060-862f-11e6-8897-2359a58ac7a5,Russia,ZmFiZTA2MC04NjJmLTExZTYtODg5Ny0yMzU5YTU4YWM3YTU.


In [13]:
def get_site_id(article_id):
        site_id = ""
        try:
            response = urllib2.urlopen("https://session-user-data.webservices.ft.com/v1/livefyre/get_lf_bootstrap?uuid="+article_id)
            data = json.loads(response.read())
            if data['url'] is not None:
                site_id = data['url']
        except Exception as e:
            print e
            
        return site_id

In [14]:
def add_livefyre_id_to_ft_id():
    chunks  = pd.read_csv("ft_id_with_livefyre_id.csv",index_col=0,chunksize=1000)

    for i,chunk in enumerate(chunks):
        if i>2:
            chunk['siteId']=chunk.ft_hash.apply(lambda x: get_site_id(x))
            chunk.to_csv("articleId_livefyreId_siteId"+str(i)+".csv")

In [15]:
def shorten_livefyre_id(x):
    livefyre_id =""
    livefyre_id =  "/".join(x.split("/")[5:7])
    return livefyre_id
                                
def clean_livefyre_id(filename):
    df = pd.read_csv("ftId_livefyreId/"+filename,index_col=0)
    print filename
    df['siteId'] = df['siteId'].astype(str)
    df = df[df['siteId'].str.contains("https")]
    
    df['livefyre_id_short'] = df['siteId'].apply(lambda x: shorten_livefyre_id(x))
    df = df[["ft_hash","topic","livefyre_id_short"]]
    return df

In [16]:
def clean_ft_id_livefyre_id():
    filelist = [f for f in listdir("ftId_livefyreId/") if isfile(join("ftId_livefyreId/", f))]
    df = pd.DataFrame()
    for filename in filelist:
        df1 = clean_livefyre_id(filename)
        df = pd.concat([df,df1],axis=0)
    df.to_csv("ft_id_livefyre_id.csv")   

In [17]:
def read_json_and_return_object(livefyre_id):
    
    url = "https://data.livefyre.com/bs3/v3.1/ft.fyre.co/"+livefyre_id+"/init"
    response = urllib2.urlopen(url)
    data = json.loads(response.read())
    return data

In [18]:
def unicodize(x):
    if isinstance(x, float):
        x = str(x)
    return x.encode("utf-8")

def create_df_from_json_content(data):
    df = pd.DataFrame()
    if 'headDocument' in data:
        data_content_norm = json_normalize(data["headDocument"]['content'])
        df = pd.DataFrame(data_content_norm)
        df.rename(columns={'collectionId':'collection_id','content.bodyHtml': 'content',
                          'content.authorId':'author_id',
                           'content.parentId':'parent_id',
                          'content.createdAt':'created_at'}, inplace=True)
        columns = ['collection_id','author_id','created_at','parent_id','content']
        if 'content.annotations.likedBy' in df.columns:
            df.rename(columns={'content.annotations.likedBy':'liked_by'},inplace=True)
            columns += ['liked_by']
        df = df[columns]
        df['content'] = df['content'].apply(lambda x : unicodize(x))
        df['content'] = df['content'].apply(cleanhtml)
    return df


In [19]:
def create_df_from_json_authors(data):
    df =pd.DataFrame()
    if 'headDocument' in data:
        df = pd.DataFrame(data["headDocument"]['authors'])
        df = df.T
        df = df.reset_index()
        df = df[['id','displayName']]
    return df
    
def get_article_title(data):
    if data['headDocument']['collectionSettings']['title'] is not None:
        return data['headDocument']['collectionSettings']['title']
    else:return ""

In [20]:
def save_livefyre_comments(topic_name):
    df = pd.read_csv("ft_id_livefyre_id.csv",index_col=0)
    df = df[df.topic==topic_name]
    live_fyre_hash = df.livefyre_id_short.tolist()
    print topic_name
    df_content = pd.DataFrame()
    df_authors = pd.DataFrame()
    for hash_livefyre in live_fyre_hash:
        
        try:
            data = read_json_and_return_object(hash_livefyre)
            df_content = pd.concat([df_content,create_df_from_json_content(data)],axis=0)
            df_authors = pd.concat([df_authors,create_df_from_json_authors(data)],axis=0)
            

        except Exception as e:
            print e
    df_content.to_csv(DATA_PATH+topic_name+"_content.csv" ,encoding='utf-8')
    df_authors.to_csv(DATA_PATH+topic_name+"_authors.csv", encoding='utf-8')
    print "files ok : "+topic_name


In [21]:
for keysword in KEY_WORDS:
    save_livefyre_comments(keysword)

Syria
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
list index out of range
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
list index out of range
list index out of range
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
list index out of range
list index out of range
list index out of range
list index out of range
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
list index out of range
HTTP Error 404: NOT FOUND
list index out of range
HTTP Error 404: NOT FOUND
HTTP Error 404: NOT FOUND
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
HTTP Error 404: NOT FOUND
list index out of range
list index out

In [22]:
df = pd.read_csv("Afd_content.csv",index_col=0)
group_content = df.groupby('author_id').count()


authors = pd.read_csv("Afd_authors.csv",index_col=0)

authors = authors.drop_duplicates('id')

group_content = group_content.sort_values(by='collection_id',ascending=False)
group_content= group_content.reset_index()
group_content.head()

groupy = pd.merge(group_content,authors,left_on='author_id',right_on='id',how='inner')

In [23]:
groupy

,author_id,collection_id,content,created_at,liked_by,parent_id,id,displayName
0,7958819@ft.fyre.co,30,30,30,4,14,7958819@ft.fyre.co,Musso
1,1457606@ft.fyre.co,23,23,23,7,21,1457606@ft.fyre.co,Pastaneta
2,190143@ft.fyre.co,14,14,14,3,7,190143@ft.fyre.co,Night Watchman
3,10270805@ft.fyre.co,14,14,14,4,14,10270805@ft.fyre.co,WallStrtKnight
4,10171768@ft.fyre.co,11,11,11,5,8,10171768@ft.fyre.co,Mostly Harmless
5,218173@ft.fyre.co,10,10,10,2,9,218173@ft.fyre.co,Egmont
6,4137934@ft.fyre.co,9,9,9,1,5,4137934@ft.fyre.co,genauer
7,5384061@ft.fyre.co,9,9,9,0,0,5384061@ft.fyre.co,Distant observer
8,275930@ft.fyre.co,9,9,9,7,8,275930@ft.fyre.co,tasdk
9,2000154855@ft.fyre.co,8,8,8,7,8,2000154855@ft.fyre.co,Markus
